In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
print(np.__file__)
import tensorflow as tf
import tensorflow_datasets as tfds
import warnings
warnings.filterwarnings('ignore')
print(tf.__version__)

/ccs/home/fwang2/.conda/envs/wm1/lib/python3.6/site-packages/numpy/__init__.py
2.1.0


In [4]:
nodes = !echo $(cat ${LSB_DJOB_HOSTFILE} | sort | uniq | grep -v login | grep -v batch) 
nodes

['h34n18']

In [5]:
!ifconfig | grep -Eo 'inet (addr:)?([0-9]*\.){3}[0-9]*' | grep -Eo '([0-9]*\.){3}[0-9]*' | grep -v '127.0.0.1'
!ifconfig | sed -En 's/127.0.0.1//;s/.*inet (addr:)?(([0-9]*\.){3}[0-9]*).*/\2/p'
!hostname
!echo $PMIX_RANK

Infiniband hardware address can be incorrect! Please read BUGS section in ifconfig(8).
Infiniband hardware address can be incorrect! Please read BUGS section in ifconfig(8).
h34n18
0


In [6]:
# Set up TF_CONFIG on this rank
import json
import os
import subprocess

nodes_list = str(nodes[0]).split(' ')
comp_nodes = nodes_list[:]
for i in range(len(nodes_list)):
    comp_nodes[i] += ".summit.olcf.ornl.gov:2222"
print(comp_nodes)

['h34n18.summit.olcf.ornl.gov:2222']


In [7]:
import json
import os
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': comp_nodes
    }, 
    'task': {'type': 'worker', 'index': 0}
})

In [8]:
# !jsrun -n2 -a1 -g6 -c42 python setup_tfconfig.py
!echo $TF_CONFIG
!nvidia-smi

{"cluster": {"worker": ["h34n18.summit.olcf.ornl.gov:2222"]}, "task": {"type": "worker", "index": 0}}
Fri Jun 26 19:44:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.116.00   Driver Version: 418.116.00   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   30C    P0    49W / 300W |  15751MiB / 16130MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| N/A   32C    P0    49W / 300W |    599MiB / 16130MiB |      0%  

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU')]